In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import json
%pylab
import multiprocessing
import scipy.optimize
from scipy.interpolate import interp1d
from scipy.integrate import odeint
from scipy.optimize import fsolve

plt.clf()
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
gs = GridSpec(2, 8)
fig = plt.gcf()
fig.set_tight_layout({'pad':0})
fig.set_size_inches((5, 2.825))
#plt.rcParams['figure.constrained_layout.use'] = True


Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
## Diagram
aTc = array([1.e-03, 1.e-02, 1.e-01, 3.e-01, 1.e+00, 3.e+00, 1.e+01, 1.e+02])

colors = ['#000fff','#2b0cd7','#550aaf','#6a099b','#800887','#aa055f','#bf044b','#d40237']
colors = flipud(['#000fff','#150eeb','#2b0cd7','#6a099b','#aa055f','#bf044b','#d40237','#ea0123'])



In [91]:
## Intensity Distribution

ax4 = plt.subplot(gs[:, :2])

aTcS = []
for i in range(8):
    if aTc[i] >= 1:
        aa = str(int(aTc[i]))
    else:
        aa = str(aTc[i])
    aTcS.append((aa)+' ng/uL')

try:
    IntensityList
except:
    IntensityList = []
    for i in range(8):
        d0 = genfromtxt('RawData/distribution_intensity_exp_'+str(i)+'.csv')
        IntensityList.append(d0)

for i in range(8):
    a, b = histogram(IntensityList[i], logspace(1, 5, 55)/log(10), density=True)
    
    print(aTcS[i]+' N = '+str(len((abs(diff(log(IntensityList[i])))>0.2).nonzero()[0]))+'\t\t'+str(len(((diff(log(IntensityList[i])))>0.25).nonzero()[0])))
    
    fill_between(log(b[:-1])/log(10), a/max(a)+i/0.9, i/0.9, step="pre", color=colors[i], alpha=0.05, zorder=(7-i))
    step(log(b[:-1])/log(10), a/max(a)+i/0.9, color=colors[i], zorder=(7-i)) 
xlabel('Intensity')

yticks(arange(8)/0.9, aTcS)


0.001 ng/uL N = 1774		889
0.01 ng/uL N = 13251		6665
0.1 ng/uL N = 16776		8522
0.3 ng/uL N = 43831		22460
1 ng/uL N = 39533		19385
3 ng/uL N = 65754		35018
10 ng/uL N = 64362		32827
100 ng/uL N = 43805		22246


([<matplotlib.axis.YTick at 0x7ff175e68a30>,
 [Text(0, 0, '0.001 ng/uL'),
  Text(0, 0, '0.01 ng/uL'),
  Text(0, 0, '0.1 ng/uL'),
  Text(0, 0, '0.3 ng/uL'),
  Text(0, 0, '1 ng/uL'),
  Text(0, 0, '3 ng/uL'),
  Text(0, 0, '10 ng/uL'),
  Text(0, 0, '100 ng/uL')])

In [48]:
#Intensity vs. aTc
ax1 = plt.subplot(gs[0, 2:5])

xlabel('aTc concentration (ng/mL)')
ylabel('Fold-change')


try:
    IntensityList
except:    
    IntensityList = []
    for i in range(8):
        d0 = genfromtxt('RawData/distribution_intensity_exp_'+str(i)+'.csv')
        IntensityList.append(d0)
Int = []
Int_err = []
        
maxInt=mean(IntensityList[7])
for i in range(8):
    errorbar(log(aTc[i])/log(10), (mean(IntensityList[i]))/maxInt, yerr=(std(IntensityList[i]))/sqrt(20)/maxInt, 
             color=colors[i], marker='o', mec='#231f20', mew=0.25, mfc=colors[i], ms=2,
             elinewidth=0.5, capsize=1)
semilogy()
xticks([-3, -2, -1, 0, 1, 2], [0, 0.01, 0.1, 1, 10, 100])
#yticks([])
xlim((-3.5, 3))
axin1 = ax1.inset_axes([0.6, 0.125, 0.35, 0.45])   
axin1.set_xlabel('aTc', fontsize=4, labelpad=1)
axin1.set_ylabel('FC', fontsize=4, labelpad=1)
axin1.set_xticks([0, 5, 10])
axin1.tick_params(labelsize=4)

for i in range(8):
    axin1.plot(log(aTc[i])/log(10), mean((IntensityList[i]))/maxInt,
             color='#ffffff', marker='o', mec='#231f20', mew=0.25, mfc=colors[i], ms=1.5, zorder=10)
axin1.spines['bottom'].set_position('zero')    
axin1.set_ylim((-5000/maxInt, 15000/maxInt))
axin1.set_xticks([-3, 0, 2])
axin1.set_xticklabels([0, 1, 100])


[Text(0, 0, '0'), Text(0, 0, '1'), Text(0, 0, '100')]

In [52]:
## Plasmid loss

ax2 = plt.subplot(gs[0, 5:])

nCellsList = []
elongated = []
grow = []
for i in range(8):
    d0 = genfromtxt('RawData/detectedcells_exp_'+str(i)+'.csv')
    nCellsList.append(d0)
    d0 = genfromtxt('RawData/elongated'+str(i)+'.csv')
    elongated.append(d0)
    d0 = genfromtxt('RawData/positivegrowth'+str(i)+'.csv')
    grow.append(d0)
    
avgFactor = 30
for i in range(8):
    nCells = nCellsList[i] - elongated[i]
    nCells = grow[i]    
    maxID = argmax(nCells)
    try:
        minID = ((array(nCells[maxID:])/nCells[maxID])>0.08).nonzero()[0][-1]
    except:
        minID = -1
    while (minID-maxID) % avgFactor != 0:
        minID -= 1        
    if minID - maxID > 1500:
        minID = maxID + 1500 
    xdata = arange(minID-maxID)
    ydata = (array(nCells[maxID:minID])/nCells[maxID])
    
    plot(xdata[:-15], ydata[:-15], lw=0.25, mew=0, color=colors[i], alpha=0.25)
    plot(hstack((0, xdata.reshape(-1, avgFactor).mean(1))), 
         hstack((1, ydata.reshape(-1, avgFactor).mean(1))), 
         label=aTcS[i], lw=1, color=colors[i])


xlim((0, 1750)) 
ylim((0.025, 1.1))
semilogy()
#legend(fontsize=4)
xlabel('Time (mins)')
ylabel('Viable cells (%)')

Text(0, 0.5, 'Viable cells (%)')

In [33]:
## Elongated / Slow growing

ax3 = plt.subplot(gs[1, 5:])

avgFactor = 30

for i in range(8):
    nCells = nCellsList[i] - elongated[i]
    nCells = grow[i]    
    maxID = argmax(nCells)
    try:
        minID = ((array(nCells[maxID:])/nCells[maxID])>0.15).nonzero()[0][-1]
    except:
        minID = len(nCells)
    
    while (minID-maxID) % avgFactor != 0:
        minID -= 1        
    if minID - maxID > 1500:
        minID = maxID + 1500
    print(i, minID, maxID, minID-maxID)    
    xdata = arange(minID-maxID)
    ydata = array(elongated[i][maxID:minID])/array(nCells[maxID:minID])
    plot(xdata[:-15], ydata[:-15], lw=0.25, mew=0, color=colors[i], alpha=0.25)    
    plot(hstack((0, xdata.reshape(-1, avgFactor).mean(1))), 
         hstack((0, ydata.reshape(-1, avgFactor).mean(1))), 
        lw=1, color=colors[i], label=aTcS[i])
    
#legend(fontsize=4)
xlim((0, 1750))


xlabel('Time (mins)')
ylabel('Non-growing cells (%)')

0 326 116 210
1 943 133 810
2 1022 62 960
3 915 75 840
4 1604 104 1500
5 1705 205 1500
6 1597 97 1500
7 1634 134 1500


Text(0, 0.5, 'Non-growing cells (%)')

In [92]:
ax4 = plt.subplot(gs[1, 2:5])

try:
    GRList
except:
    GRList = []
    for i in range(8):
        d0 = genfromtxt('RawData/distribution_growthrate_exp_'+str(i)+'.csv')
        GRList.append((array(d0)))


for i in [0, 1, 2, 3, 4]:
    a, b = histogram(log(array(GRList[i])), 30, density=True)
    #fill_between((b[:-1]), a/max(a), i/0.9, step="pre", color='#f0f0f0', zorder=(7-i))
    fill_between((b[:-1]), a/max(a), zorder=(6-i), color=colors[i], alpha=0.)
    plot((b[:-1]), a/max(a), zorder=(6-i), color=colors[i],label=aTcS[i]) 
    #GRhigh = hstack((array(GRList[6]),
#                 array(GRList[7])))
#a, b = histogram(log((GRhigh)/log(2)*43), 25, density=True)
#fill_between((b[:-1]), a/max(a), i/0.9, step="pre", color='#f0f0f0', zorder=(7-i))
#plot((b[:-1]), a/max(a), zorder=(7-i), label='10+')
    
legend(fontsize=4)
xlabel('log[growth rate]')
ylabel('Rel. Density')
'''
for i in range(8):
    a, b = histogram(array(GRList[i]), logspace(-2.35, -1.375, 51), density=True)
    errorbar(i, mean(GRList[i]), std(GRList[i]), marker='o', color='#cc333f')
ylim((0, 0.03))    
'''
xlim((-5.35, -3.25))
ylim((0, 1.1))
#violinplot((GRList[4:]))


(0.0, 1.1)

In [110]:
for i in range(8):
    print(aTcS[i], mean(GRList[i]), len(((abs(diff(GRList[i]))>0)).nonzero()[0]))

0.001 ng/uL 0.017699052964690206 1455
0.01 ng/uL 0.01638409354323867 10276
0.1 ng/uL 0.017225233427411998 13800
0.3 ng/uL 0.01575131855009643 33156
1 ng/uL 0.01475840158381491 35277
3 ng/uL 0.015075199214408362 71885
10 ng/uL 0.014852153839449319 81511
100 ng/uL 0.013593768820250167 59757


In [50]:
hist(log(array(GRList[2])), 100, histtype='step', density=True)
hist(log(array(GRList[3])), 100, histtype='step', density=True)
hist(log(array(GRList[4])), 100, histtype='step', density=True)

(array([0.25667979, 0.27877753, 0.33539807, 0.39760203, 0.31691776,
        0.33079765, 0.29403362, 0.32832051, 0.29780832, 0.34141401,
        0.34322272, 0.35234492, 0.40019714, 0.41918861, 0.37404946,
        0.44616201, 0.36087732, 0.39123222, 0.37959355, 0.41568915,
        0.44521833, 0.40629171, 0.40751062, 0.48603231, 0.44403874,
        0.51237659, 0.53128942, 0.57214272, 0.52185266, 0.51705565,
        0.50667521, 0.59738604, 0.55535315, 0.63470055, 0.54147325,
        0.67972175, 0.67115002, 0.67622228, 0.65333815, 0.64048056,
        0.64130628, 0.7064199 , 0.69049538, 0.69930302, 0.70767814,
        0.68734979, 0.79602978, 0.728439  , 0.7210862 , 0.7466834 ,
        0.75419349, 0.81010627, 0.77766742, 0.77794266, 0.80141659,
        0.81226886, 0.85390855, 0.80994899, 0.77200537, 0.77566211,
        0.76720835, 0.82953026, 0.7732636 , 0.71349747, 0.75509784,
        0.72195123, 0.73972379, 0.67453153, 0.62502787, 0.63831797,
        0.63827865, 0.59919475, 0.55759438, 0.46

In [220]:
fig = plt.gcf()
plt.tight_layout()
fig.set_tight_layout({'pad':2})
fig.set_size_inches((5.25, 3))
plt.savefig('Fig3.pdf')
plt.savefig('Fig3.svg')


In [108]:
len(((abs(diff(GRList[4]))>0)).nonzero()[0])

35277

In [793]:
SP2dict['P2']['Fit']

[249.52454488329013,
 28.221417495400516,
 2.812684462144393,
 0.29099569822711435]

In [797]:
SP2dict['BlankCtrl']

'Fluor_w37_H_GL002_Background.csv'

In [493]:
for i, p in enumerate(plasmidList):
    atc = SP2dict['aTc']
    fitparams = SP2dict[p]['Fit']
    errorbar(atc[1:], (array(SP2dict[p]['GMax_mean'][1:])-LOW_LEVEL)/(HIGH_LEVEL - LOW_LEVEL), yerr=array(SP2dict[p]['GMax_std'][1:])/(HIGH_LEVEL-LOW_LEVEL), color=colorList[i], marker='o', ms=2,  mew=0.25, mec='#231f20', lw=0, elinewidth=0.5, capsize=1)
    semilogx(atcH, (exp(loghill(log(atcH), *fitparams))-LOW_LEVEL)/(HIGH_LEVEL - LOW_LEVEL), color=colorList[i], label='Nc = '+str(Ndict[p]))

In [698]:
figure()
x = logspace(-1.5, 3, 100)
for i, n in enumerate([1, 50, 30, 170, 270]):
    xx = 5/(0.27**2+x**(-2))
    ff = fc(xx, k=1, Ns=0, bs=-13, Nc=n, bc=-13, Nns=5.*10**6)
    semilogx(x, (log(ff/(1-ff))), color=colorList[i])

In [802]:
SP2dict['P2']['Fit']

[249.52454488329013,
 28.221417495400516,
 2.812684462144393,
 0.29099569822711435]

In [799]:
LOW_LEVEL = 0
HIGH_LEVEL = 1
for i, p in enumerate(plasmidList):
    atc = SP2dict['aTc']
    fitparams = SP2dict[p]['Fit']
    errorbar(atc[1:], (array(SP2dict[p]['GMax_mean'][1:])-LOW_LEVEL)/(HIGH_LEVEL - LOW_LEVEL), yerr=array(SP2dict[p]['GMax_std'][1:])/(HIGH_LEVEL-LOW_LEVEL), color=colorList[i], marker='o', ms=2,  mew=0.25, mec='#231f20', lw=0, elinewidth=0.5, capsize=1)
    semilogx(atcH, (exp(loghill(log(atcH), *fitparams))-LOW_LEVEL)/(HIGH_LEVEL - LOW_LEVEL), color=colorList[i], label='Nc = '+str(Ndict[p]))

In [12]:
fc(xx, k=1, Ns=0, bs=-17, Nc=n, bc=-17, Nns=5.*10**6)

TypeError: 'float' object is not callable

In [14]:
fc

0.6251711330475422

In [245]:
import numpy as np
import matplotlib.pyplot as plt

# example data
x = np.arange(0.1, 4, 0.5)
y = np.exp(-x)

# example error bar values that vary with x-position
error = 0.1 + 0.2 * x

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(x, y, yerr=error, fmt='-o')
ax0.set_title('variable, symmetric error')

# error bar values w/ different -/+ errors that
# also vary with the x-position
lower_error = 0 * error
upper_error = error
asymmetric_error = [lower_error, upper_error]

ax1.errorbar(x, y, yerr=asymmetric_error, fmt='o')
ax1.set_title('variable, asymmetric error')
ax1.set_yscale('log')
plt.show()

In [389]:
i = 0
plot(array(nCellsList[i]) - array(grow[i]))

In [90]:
i=1
plot(((IntensityList[i][:1000])))
idd = (abs(diff(log(IntensityList[i][:1000])))>0.1).nonzero()[0]
plot(arange(len(IntensityList[i]))[:1000][idd], IntensityList[i][:1000][idd],'or')

In [83]:
len(idd)

24

In [67]:
sub
plot(diff(log(IntensityList[0])[:1000]))

33022